# Predicting Customer Churn with Machine Learning 🏦💰

<br>

**Table of Contents**

- [Introduction](#Introduction)
- [Libraries](#Libraries)
- [Default Parameters](#Default-Parameters)
- [A Quick Look at our Data](#A-Quick-Look-at-our-Data)
- [Creating a Test Set](#Creating-a-Test-Set)
- [Exploratory Data Analysis](#Exploratory-Data-Analysis)
- [Data Preprocessing](#Data-Preprocessing)
    - [Feature Selection](#Feature-Selection) <br>
    - [Encoding Categorical Features](#Encoding-Categorical-Features) <br>
    - [Scaling](#Scaling) <br>
    - [Addressing Class Imbalance](#Addressing-Class-Imbalance) 
- [Building Machine Learning Models](#Building-Machine-Learning-Models)
    - [Baseline Models](#Baseline-Models) <br>
    - [Model Tuning](#Model-Tuning) <br>
- [Results](#Results)
- [Evaluating the Test Set](#Evaluating-the-Test-Set)
- [Bibliography](#Bibliography)
- [Conclusions](#Conclusions)


<br>

# Introduction

<br>

# Libraries

<br>

# Default Parameters

<br>

# A Quick Look at our Data

<br>

# Creating a Test Set

<br>

# Exploratory Data Analysis

<br>

# Data Preprocessing

<br>

## Feature Selection

<br>

## Encoding Categorical Features

<br>

## Scaling

<br>

## Addressing Class Imbalance

<br>

# Building Machine Learning Models

<br> 

## Baseline Models

<br>

## Model Tuning

<br>

# Results

<br>

# Evaluating the Test Set

<br>

# Bibliography

<br>

# Conclusions